# IN3050/IN4050 - Week 2
## Representations

### 1. ![Naming_Question](EA_Terms.png)

Name the terms shown in the picture above.

# Add your solution here
### A: Locus
### B: Allele
### C: Gene
### D: Genotype
### E: Phenotype

### 2. Mention some of the most common representations of genomes.

Genomes are represented in binary, integers, permutation, tree, and real numbers

### 3. Perform a mutation operation on the representations given below.

binary = $[1, 0, 1, 1]$;
integer = $[4, 2, 4, 1]$;
real_valued = $[2.53, 1.42, 3.14, 1.68]$;
permutation = $[3, 4, 1, 2]$

In [61]:
b   = [1, 0, 1, 1]
b_p = [1, 1, 1, 1]

i   = [4, 2, 4, 1]
i_p   = [0, 2, 4, 1]

rv   = [2.53, 1.42, 3.14, 1.68] 
rv_p = [3.14, 1.68, 2.53, 1.42]

### 4. Given the sequences (2,4,7,1,3,6,8,9,5) and (5,9,8,6,2,4,1,3,7). Implement these algorithms to create a new pair of solutions: 

#### a. Partially mapped crossover (PMX)

In [62]:
# Add your solution here
import numpy as np
from typing import Iterable
np.random.seed(42)   

def pmx(p1: Iterable, p2: Iterable):
    def procedure(p1: Iterable, p2: Iterable):
        p1, p2 = np.array(p1), np.array(p2)
        # Step 1
        idx1 = np.random.randint(0, len(p1))
        idx2 = np.random.randint(0, len(p1))
        
        while idx1 == idx2:
            idx2 = np.random.randint(0, len(p1))
        
        if idx1 > idx2:
            idx2, idx1 = idx1, idx2
        
        segment = slice(idx1, idx2+1)
        # segment = slice(3,7) # Sanity check (lecture 3, slide 56)
        
        child = np.zeros(len(p1))
        child[segment] = p1[segment]
        
        # Step 2
        for gene1 in p1[segment]:
            gene1_idx = np.where(p1 == gene1)
            gene2 = p2[gene1_idx]
            
            if gene2 in child:
                continue
            
            gene1_in_p2_idx = np.where(p2 == gene1)
            while gene1 in p2[segment]:
                gene1 = p1[gene1_in_p2_idx]
                gene1_in_p2_idx = np.where(p2 == gene1)
                    
            child[gene1_in_p2_idx] = gene2
            
        # Step 3
        for idx, gene in enumerate(child):
            if gene == 0:
                child[idx] = p2[idx]
                
        return child    
    
    child1 = procedure(p1, p2)
    child2 = procedure(p2, p1)
    
    return child1, child2

p1 = (2,4,7,1,3,6,8,9,5)
p2 = (5,9,8,6,2,4,1,3,7)

# Sanity check (lecture 3, slide 56)
# p1 = (1,2,3,4,5,6,7,8,9)
# p2 = (9,3,7,8,2,6,5,1,4)

child1, child2 = pmx(p1, p2)
print(f'{child1 = }')
print(f'{child2 = }')

child1 = array([5., 9., 4., 1., 3., 6., 8., 2., 7.])
child2 = array([9., 6., 7., 8., 2., 4., 1., 3., 5.])


#### b. Order crossover

In [63]:
# Add your solution here
import numpy as np
from typing import Iterable
np.random.seed(42)   

def order_crossover(p1: Iterable, p2: Iterable):
    def procedure(p1: Iterable, p2: Iterable):
        p1, p2 = np.array(p1), np.array(p2)
        # Step 1
        idx1 = np.random.randint(0, len(p1))
        idx2 = np.random.randint(0, len(p1))
        
        while idx1 == idx2:
            idx2 = np.random.randint(0, len(p1))
        
        if idx1 > idx2:
            idx2, idx1 = idx1, idx2
        
        segment = slice(idx1, idx2+1)
        # segment = slice(3,7) # Sanity check (lecture 3, slide 61)
        
        child = np.zeros(len(p1))
        child[segment] = p1[segment]
        
        # Step 2
        child = np.roll(child, -idx2 - 1)
        for gene in np.roll(p2, -idx2 - 1):
            if gene not in child:
                idx = np.where(child == 0)[0][0]
                child[idx] = gene
        
        child = np.roll(child, idx2 + 1)
        
        return child 
    
    child1 = procedure(p1, p2)
    child2 = procedure(p2, p1)
    
    return child1, child2

p1 = (2,4,7,1,3,6,8,9,5)
p2 = (5,9,8,6,2,4,1,3,7)

# Sanity check (lecture 3, slide 61)
# p1 = (1,2,3,4,5,6,7,8,9)
# p2 = (9,3,7,8,2,6,5,1,4)

child1, child2 = order_crossover(p1, p2)
print(f'{child1 = }')
print(f'{child2 = }')

child1 = array([9., 2., 4., 1., 3., 6., 8., 7., 5.])
child2 = array([7., 6., 8., 9., 2., 4., 1., 3., 5.])


#### c. Cycle crossover

##### Original (incomplete)

In [64]:
# Add your solution here
import numpy as np
from typing import Iterable

def cycle_crossover(p1: Iterable, p2: Iterable):
    p1, p2 = np.array(p1), np.array(p2)
    child1, child2 = np.zeros(len(p1)), np.zeros(len(p1))
    
    for i in range(2):
        gene1 = p1[i]
        gene2 = p2[i]
        if gene1 == gene2:
            child1[i], child2[i] = gene1, gene2
            continue
        
        first = gene1
        idx1 = np.where(p1 == gene2)
        idx2 = np.where(p2 == gene2)
        
        child1[idx1] = gene2
        child2[idx2] = gene2
        
        gene1 = p1[idx1]
        gene2 = p2[idx1]
        
        print("post-prep")
        print(f'{child1 = }')
        print(f'{child2 = }\n')
        # print(f'{gene1, gene2}') 
        print('prep done')
        while gene1 != first and not np.all(child1):
            
            idx1 = np.where(p1 == gene2)
            idx2 = np.where(p2 == gene2)
            
            child1[idx1] = gene2
            child2[idx1] = gene2
            print('Child update')
            print(f'{child1 = }')
            print(f'{child2 = }\n')    
            
            gene1 = p1[idx1]
            gene2 = p2[idx1]
                
        print('While-loop Done')
        print(f'{child1 = }')
        print(f'{child2 = }\n')
        print('---------------')
            
        
        p1, p2 = p2, p1
        child1, child2 = child2, child1
    return child1, child2

# Sanity check (lecture 3, slide 63)
p1 = (1,2,3,4,5,6,7,8,9)
p2 = (9,3,7,8,2,6,5,1,4)

child1, child2 = cycle_crossover(p1, p2)
# print(f'{child1 = }')
# print(f'{child2 = }')

post-prep
child1 = array([0., 0., 0., 0., 0., 0., 0., 0., 9.])
child2 = array([9., 0., 0., 0., 0., 0., 0., 0., 0.])

prep done
Child update
child1 = array([0., 0., 0., 4., 0., 0., 0., 0., 9.])
child2 = array([9., 0., 0., 4., 0., 0., 0., 0., 0.])

Child update
child1 = array([0., 0., 0., 4., 0., 0., 0., 8., 9.])
child2 = array([9., 0., 0., 4., 0., 0., 0., 8., 0.])

Child update
child1 = array([1., 0., 0., 4., 0., 0., 0., 8., 9.])
child2 = array([1., 0., 0., 4., 0., 0., 0., 8., 0.])

While-loop Done
child1 = array([1., 0., 0., 4., 0., 0., 0., 8., 9.])
child2 = array([1., 0., 0., 4., 0., 0., 0., 8., 0.])

---------------
post-prep
child1 = array([1., 0., 0., 4., 2., 0., 0., 8., 0.])
child2 = array([1., 2., 0., 4., 0., 0., 0., 8., 9.])

prep done
Child update
child1 = array([1., 0., 0., 4., 2., 0., 5., 8., 0.])
child2 = array([1., 2., 0., 4., 0., 0., 5., 8., 9.])

Child update
child1 = array([1., 0., 7., 4., 2., 0., 5., 8., 0.])
child2 = array([1., 2., 7., 4., 0., 0., 5., 8., 9.])

Child u

##### Works and follows instructions. Might need rewrite for arbitrary number of cycles

In [65]:
import numpy as np
from typing import Iterable

def cycle_crossover(p1: Iterable, p2: Iterable):
    p1, p2 = np.array(p1), np.array(p2)
    # Step 1
    
    def id_cycle(p1: np.array, p2: np.array, start: int) -> dict:
        # Start the cycle
        allele = p1[start]
        allele_adj = p2[start]
        
        idx = np.where(p1 == allele_adj)[0][0]
        cycle = {}
        cycle[idx] = p1[idx] # Could do "... = allele_adj"
        
        ## Continue the cycle
        first = allele 
        allele = p1[idx]
        allele_adj = p2[idx]
        while allele != first:
            idx = np.where(p1 == allele_adj)[0][0]
            cycle[idx] = p1[idx]    
            allele = p1[idx]
            allele_adj = p2[idx]
            
            
        return cycle

    cycle1 = id_cycle(p1, p2, 0)
    cycle2 = id_cycle(p1, p2, 1)
    
    # child1 = p1.copy()
    # child2 = p2.copy()
    child1 = np.zeros(len(p1))
    child2 = np.zeros(len(p2))
    
    
    print(cycle1)
    print(cycle2)
    # for (idx1, allele1), (idx2, allele2) in zip(cycle1.items(), cycle2.items()):
    #     # TODO: Something wrong about child1, but this seems to follow the instructions???
    #     print(idx1, allele1)
    #     print(idx2, allele2)
    #     print()
    #     child1[idx1] = allele1
    #     child2[idx2] = allele2
        
    # Is this reduntent when doing a copy? 
    # Adding the dark grey numbers
    for idx, allele in cycle1.items():
        child1[idx] = allele 
        child2[idx] = p2[idx]
        
    # Adding the white numbers
    for idx, allele in cycle2.items():
        child2[idx] = allele 
        child1[idx] = p1[idx]
        
    # Adding the red rumbers (a cycle of 1)
    for idx in np.where(p1 == p2)[0]:
        child1[idx] = p1[idx]
        child2[idx] = p2[idx]
            
    
    return child1, child2
# Sanity check (lecture 3, slide 63)
p1 = (1,2,3,4,5,6,7,8,9)
p2 = (9,3,7,8,2,6,5,1,4)

# cycle_crossover(p1, p2)
child1, child2 = cycle_crossover(p1, p2)
print(f'{child1 = }')
print(f'{child2 = }')

{8: 9, 3: 4, 7: 8, 0: 1}
{2: 3, 6: 7, 4: 5, 1: 2}
child1 = array([1., 2., 3., 4., 5., 6., 7., 8., 9.])
child2 = array([9., 2., 3., 8., 5., 6., 7., 1., 4.])


##### Hack

In [66]:
import numpy as np
from typing import Iterable

def cycle_crossover(p1: Iterable, p2: Iterable):
    p1, p2 = np.array(p1), np.array(p2)
    # Step 1

    # Start the cycle
    allele = p1[1]
    allele_adj = p2[1]
    
    idx = np.where(p1 == allele_adj)[0][0]
    cycle_alt = [idx]
    
    ## Continue the cycle
    first = allele 
    allele = p1[idx]
    allele_adj = p2[idx]
    while allele != first:
        idx = np.where(p1 == allele_adj)[0][0]
        cycle_alt.append(idx)   
        allele = p1[idx]
        allele_adj = p2[idx]
        
    
    child1 = p1.copy()
    child2 = p2.copy()
    
    for idx in cycle_alt:
        child1[idx] = p2[idx]
        child2[idx] = p1[idx]
   
    
    return child1, child2
# Sanity check (lecture 3, slide 63)
p1 = (1,2,3,4,5,6,7,8,9)
p2 = (9,3,7,8,2,6,5,1,4)

# cycle_crossover(p1, p2)
child1, child2 = cycle_crossover(p1, p2)
print(f'{child1 = }')
print(f'{child2 = }')

child1 = array([1, 3, 7, 4, 2, 6, 5, 8, 9])
child2 = array([9, 2, 3, 8, 5, 6, 7, 1, 4])


#### Test crossovers

In [67]:
a = [2, 4, 7, 1, 3, 6, 8, 9, 5]
b = [5, 9, 8, 6, 2, 4, 1, 3, 7]
c, d = pmx(a, b)
e, f = order_crossover(a, b)
g, h = cycle_crossover(a, b)
print(f"Parents: {a} and {b}")
print(f"Children after PMX: {c} and {d}")
print(f"Children after Order Crossover: {e} and {f}")
print(f"Children after Cycle Crossover: {g} and {h}")

Parents: [2, 4, 7, 1, 3, 6, 8, 9, 5] and [5, 9, 8, 6, 2, 4, 1, 3, 7]
Children after PMX: [5. 9. 7. 1. 3. 6. 8. 2. 4.] and [2. 4. 7. 8. 9. 6. 1. 3. 5.]
Children after Order Crossover: [8. 6. 2. 1. 3. 4. 7. 5. 9.] and [7. 9. 8. 6. 2. 4. 1. 3. 5.]
Children after Cycle Crossover: [5 9 8 6 2 4 1 3 7] and [2 4 7 1 3 6 8 9 5]
